In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

     |████████████████████████████████| 274 kB 7.8 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=6d910925fe6e04e7fc1716f844c28063e08161465fb24f55d1136fa1f72e7a71
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 4.0 MB/s 
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 74 kB 4.4 MB/s 
     |████████████████████████████████| 145 kB 77.0 MB/s 
     |████████████████████████████████| 130 kB 73.8 MB/s 
     |████████████████████████████████| 748 kB 68.4 MB/s 
     |███████████████████████████████

In [ ]:
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git


  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-udyyeby9
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-udyyeby9
     |████████████████████████████████| 55 kB 1.7 MB/s 
     |████████████████████████████████| 74 kB 2.5 MB/s 
     |████████████████████████████████| 145 kB 50.8 MB/s 
     |████████████████████████████████| 130 kB 36.6 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 748 kB 43.5 MB/s 
     |████████████████████████████████| 843 kB 57.2 MB/s 
     |████████████████████████████████| 596 kB 57.8 MB/s 
     |████████████████████████████████| 112 kB 78.0 MB/s 
  Created wheel for detectron2: filename=detectron2-0.6-cp37-cp37m-linux_x86_64.whl size=5367407 sha256=c429608d9172f80424d03968a78651972aab45ce781d03ffcb631d5462d5a7ea
  Stored in directory: /tmp/pip-ephem-wheel-cache-p3_1iaur/wheels/07/dc/32/0322cb484dbefab8b9366bfedbaff5060ac7

In [1]:
datapath = '/content/drive/MyDrive/PyPSA_Africa_images/datasets'

import sys
sys.argv = ['']

In [2]:
# Add pypsa africa tool
import os
os.chdir('/content/drive/MyDrive/PyPSA_Africa_images/unbiased-teacher/')

import sys
sys.path.append('/content/drive/MyDrive/PyPSA_Africa_images/detect_energy/src/')

In [3]:
import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import get_cfg
from detectron2.engine import default_argument_parser, default_setup, launch

from ubteacher import add_ubteacher_config
from ubteacher.engine.trainer import UBTeacherTrainer, BaselineTrainer

# hacky way to register
from ubteacher.modeling.meta_arch.rcnn import TwoStagePseudoLabGeneralizedRCNN
from ubteacher.modeling.proposal_generator.rpn import PseudoLabRPN
from ubteacher.modeling.roi_heads.roi_heads import StandardROIHeadsPseudoLab
import ubteacher.data.datasets.builtin

from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel

# import some of our own methods
from utils.detectron_utils import eval_predictor
from utils.image_utils import get_true_images

In [4]:
def main(args):
    cfg = setup(args)

    if cfg.SEMISUPNET.Trainer == "ubteacher":
        Trainer = UBTeacherTrainer
    elif cfg.SEMISUPNET.Trainer == "baseline":
        Trainer = BaselineTrainer
    else:
        raise ValueError("Trainer Name is not found.")

    trainer = Trainer(cfg)    
    trainer.train()
    
    return None


In [5]:
def setup(args):
    """
    Create configs and perform basic setups.
    """
    cfg = get_cfg()
    # print(cfg)
    add_ubteacher_config(cfg)
    # print(cfg)
    # cfg.merge_from_file(args.config_file)
    # cfg.merge_from_list(args.opts)
    # cfg.freeze()
    default_setup(cfg, args)
    return cfg


In [6]:
args = default_argument_parser().parse_args()
cfg = setup(args)

[01/22 16:17:15 detectron2]: Rank of current process: 0. World size: 1
[01/22 16:17:17 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
numpy                   1.19.5
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.1
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0+cu111 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla P100-PCIE-16GB (arch=6.0)
Driver version          460.32.03
CUDA_HOME               /usr/local/cuda
Pillow                  7.1.2
torchvision             0.11.1+cu111 @/usr/local/lib/python3.7/dist-packages/torchvision
torchvision arch fl

In [7]:
cfg = setup(args)
if cfg.SEMISUPNET.Trainer == "ubteacher":
    Trainer = UBTeacherTrainer
elif cfg.SEMISUPNET.Trainer == "baseline":
    Trainer = BaselineTrainer
else:
    raise ValueError("Trainer Name is not found.")


[01/22 16:17:20 detectron2]: Rank of current process: 0. World size: 1
[01/22 16:17:21 detectron2]: Environment info:
----------------------  ----------------------------------------------------------------
sys.platform            linux
Python                  3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0]
numpy                   1.19.5
detectron2              0.6 @/usr/local/lib/python3.7/dist-packages/detectron2
Compiler                GCC 7.3
CUDA compiler           CUDA 11.1
detectron2 arch flags   3.7, 5.0, 5.2, 6.0, 6.1, 7.0, 7.5, 8.0, 8.6
DETECTRON2_ENV_MODULE   <not set>
PyTorch                 1.10.0+cu111 @/usr/local/lib/python3.7/dist-packages/torch
PyTorch debug build     False
GPU available           Yes
GPU 0                   Tesla P100-PCIE-16GB (arch=6.0)
Driver version          460.32.03
CUDA_HOME               /usr/local/cuda
Pillow                  7.1.2
torchvision             0.11.1+cu111 @/usr/local/lib/python3.7/dist-packages/torchvision
torchvision arch fl

In [8]:
import os
import time
import logging
import torch
from torch.nn.parallel import DistributedDataParallel
from fvcore.nn.precise_bn import get_bn_modules
import numpy as np
from collections import OrderedDict

import detectron2.utils.comm as comm
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.engine import DefaultTrainer, SimpleTrainer, TrainerBase
from detectron2.engine.train_loop import AMPTrainer
from detectron2.utils.events import EventStorage
from detectron2.evaluation import COCOEvaluator, verify_results, PascalVOCDetectionEvaluator, DatasetEvaluators
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.engine import hooks
from detectron2.structures.boxes import Boxes
from detectron2.structures.instances import Instances
from detectron2.utils.env import TORCH_VERSION
from detectron2.data import MetadataCatalog
from detectron2.modeling import build_model
from detectron2 import model_zoo

from ubteacher.data.build import (
    build_detection_semisup_train_loader,
    build_detection_test_loader,
    build_detection_semisup_train_loader_two_crops,
)
from ubteacher.data.dataset_mapper import DatasetMapperTwoCropSeparate
from ubteacher.engine.hooks import LossEvalHook
from ubteacher.modeling.meta_arch.ts_ensemble import EnsembleTSModel
from ubteacher.checkpoint.detection_checkpoint import DetectionTSCheckpointer
from ubteacher.solver.build import build_lr_scheduler

In [36]:
import numpy as np
import logging

from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.data.build import get_detection_dataset_dicts
from detectron2.data.common import DatasetFromList, MapDataset
from detectron2.data.samplers import TrainingSampler

from ubteacher.data.build import divide_label_unlabel

from itertools import product


def build_maxar_loader(cfg, mapper=None):
    '''
    Sets up loader for custom maxar data
    '''

    # register used datasets
    ds_names = ['fake_maxar', 'maxar']
    modes = ['train', 'val']

    for name, mode in product(ds_names, modes):

        ds_name = f'{name}_{mode}'
        json_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/{ds_name}/labels.json'
        ds_path = f'/content/drive/My Drive/PyPSA_Africa_images/datasets/{ds_name}/data/' 
    
        if ds_name in DatasetCatalog.list():
            DatasetCatalog.remove(ds_name)
            MetadataCatalog.remove(ds_name)
    
        register_coco_instances(ds_name, {}, json_path, ds_path)
    
    unlabel_dicts = get_detection_dataset_dicts(
                            ['maxar_train', 'maxar_val'],
                            filter_empty=False,
                            min_keypoints=0,
                            proposal_files=None
                        )

    label_dicts = get_detection_dataset_dicts(
                            ['fake_maxar_train', 'fake_maxar_val'],
                            filter_empty=False,
                            min_keypoints=0,
                            proposal_files=None
                        )
    
    dataset = DatasetFromList(label_dicts, copy=False)

    if mapper is None:
        mapper = DatasetMapper(cfg, True)
    dataset = MapDataset(dataset, mapper)

    sampler_name = cfg.DATALOADER.SAMPLER_TRAIN
    assert sampler_name == 'TrainingSampler', f'Only TrainingSampler has been added, instead of {sampler_name}.'

    logger = logging.getLogger(__name__)
    logger.info("Using training sampler {}".format(sampler_name))

    if sampler_name == 'TrainingSampler':
        sampler = TrainingSampler(len(dataset))


    





cfg.DATASETS.TRAIN = ['maxar_train', 'maxar_val']


loader = build_maxar_loader(cfg)


WARNING [01/22 18:53:17 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/22 18:53:17 d2.data.datasets.coco]: Loaded 762 images in COCO format from /content/drive/My Drive/PyPSA_Africa_images/datasets/maxar_train/labels.json
WARNING [01/22 18:53:17 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/22 18:53:17 d2.data.datasets.coco]: Loaded 190 images in COCO format from /content/drive/My Drive/PyPSA_Africa_images/datasets/maxar_val/labels.json
WARNING [01/22 18:53:18 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[01/22 18:53:18 d2.data.datasets.coco]: Loaded 11813 images in COCO format from /content/drive/My Drive/PyPSA_Africa_images/datasets/fake_maxar_train/labels.json
WARNING [01/22 18:53:18 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll appl

In [17]:
print(cfg.DATASETS.TRAIN)
print(cfg.DATALOADER.FILTER_EMPTY_ANNOTATIONS)
print(cfg.MODEL.ROI_KEYPOINT_HEAD.MIN_KEYPOINTS_PER_IMAGE)
print(cfg.MODEL.KEYPOINT_ON)
print(cfg.DATASETS.PROPOSAL_FILES_TRAIN)
print(cfg.MODEL.LOAD_PROPOSALS)

('coco_2017_train',)
True
1
False
()
False


In [14]:
class UBTeacherTrainer(DefaultTrainer):
    def __init__(self, cfg):
        """
        Args:
            cfg (CfgNode):
        Use the custom checkpointer, which loads other backbone models
        with matching heuristics.
        """
        cfg = DefaultTrainer.auto_scale_workers(cfg, comm.get_world_size())

        frcnn = 'faster_rcnn_R_101_FPN_3x.yaml'

        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/"+frcnn))
        cfg.INPUT.FORMAT = "RGB"
        cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/PyPSA_Africa_images/PISA_models/11_01_2022_fake_maxar_train/model_final.pth'

        '''
        model_path = '/content/drive/MyDrive/PyPSA_Africa_images/PISA_models/11_01_2022_fake_maxar_train/model_final.pth'
        duke_img_dir = '/content/drive/MyDrive/PyPSA_Africa_images/datasets/duke_val/data/'
        test_imgs = get_true_images(duke_img_dir, 20)
        eval_predictor(test_imgs, model_path=model_path)
        '''

        # create an student model
        model = self.build_model(cfg)

        # data_loader = self.build_train_loader(cfg)
        
        optimizer = self.build_optimizer(cfg, model)

        # create an teacher model
        model_teacher = self.build_model(cfg)
        self.model_teacher = model_teacher

        # For training, wrap with DDP. But don't need this for inference.
        if comm.get_world_size() > 1:
            model = DistributedDataParallel(
                model, device_ids=[comm.get_local_rank()], broadcast_buffers=False
            )

        TrainerBase.__init__(self)
        self._trainer = (AMPTrainer if cfg.SOLVER.AMP.ENABLED else SimpleTrainer)(
            model, data_loader, optimizer
        )
        self.scheduler = self.build_lr_scheduler(cfg, optimizer)

        # Ensemble teacher and student model is for model saving and loading
        ensem_ts_model = EnsembleTSModel(model_teacher, model)

        self.checkpointer = DetectionTSCheckpointer(
            ensem_ts_model,
            cfg.OUTPUT_DIR,
            optimizer=optimizer,
            scheduler=self.scheduler,
        )
        self.start_iter = 0
        self.max_iter = cfg.SOLVER.MAX_ITER
        self.cfg = cfg

        self.register_hooks(self.build_hooks())
        print('reached the return')
        return

    def train(self):
        print('totally trained')
        pass


print(cfg)
trainer = UBTeacherTrainer(cfg)    
trainer.train()


CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  RANDOM_DATA_SEED: 0
  RANDOM_DATA_SEED_PATH: dataseed/COCO_supervision.txt
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
  SUP_PERCENT: 100.0
DATASETS:
  CROSS_DATASET: False
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('coco_2017_val',)
  TRAIN: ('coco_2017_train',)
  TRAIN_LABEL: ('coco_2017_train',)
  TRAIN_UNLABEL: ('coco_2017_train',)
EMAMODEL:
  SUP_CONSIST: True
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: RGB
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
 

NameError: ignored